In [ ]:
from typing import Annotated, TypedDict, List, Dict, Any, Optional, Literal
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import nest_asyncio
import gradio as gr
import uuid
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

In [ ]:
INITIAL = "initial"
GENERATE_CLARIFICATION_QUERY = "generate_clarification_query"
CONFIRM_COMPETITORS = "confirm_competitors"
GET_COMPETITORS = "get_competitors"
GET_COMPETITORS_PROMPT = "get_competitors_prompt"
GET_REPORT = "get_report"
FINAL = "final"
THREAD_ID = str(uuid.uuid4())
LANGSMITH_PROJECT="pr-crushing-pharmacopoeia-50"

beautify_output = {
    GENERATE_CLARIFICATION_QUERY: "🤔 Understanding your business details...",
    GET_COMPETITORS: "🔍 Searching for your competitors...",
    GET_COMPETITORS_PROMPT: "📝 Preparing research queries for each competitor...",
    GET_REPORT: "📊 Researching competitors in depth (this may take a moment)...",
    FINAL: "✅ Generating your competitive analysis report...",
}

In [ ]:
class State(TypedDict):
    messages: Annotated[List[Any], add_messages]
    stage: Literal[
        INITIAL,
        GENERATE_CLARIFICATION_QUERY,
        GET_COMPETITORS,                
        GET_COMPETITORS_PROMPT,
        GET_REPORT,
        CONFIRM_COMPETITORS,
        FINAL, 
    ]
    business_info: Optional[str] = Field(description="The business info")
    competitors: Optional[List[str]] = Field(description="The competitors of the business")
    competitors_query: Optional[Dict[str, str]] = Field(description="The query to get the competitors")
    competitors_report: Optional[Dict[str, str]] = Field(description="The report of the competitors")

In [ ]:
class LLMResponse(BaseModel):
    response: str = Field(description="The response to the user's message")
    stage: Optional[Literal[
        INITIAL,
        GENERATE_CLARIFICATION_QUERY,
        GET_COMPETITORS,                
        GET_COMPETITORS_PROMPT,
        CONFIRM_COMPETITORS,
        FINAL
    ]] = Field(description="The next stage of the workflow")
    business_info: Optional[str] = Field(description="The business info")
    competitors: Optional[List[str]] = Field(description="The competitors of the business")
    competitors_query: Optional[Dict[str, str]] = Field(description="The query to get the competitors")
    competitors_report: Optional[Dict[str, str]] = Field(description="The report of the competitors")

In [ ]:

nest_asyncio.apply()
async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()  

In [ ]:
# Bind LLM to tools
tools_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)
struct_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(LLMResponse)

In [ ]:
def formatter(tool_result, state):
    """
    Converts tool-calling LLM results into structured LLMResponse format.
    This is needed because we can't use .bind_tools() AND .with_structured_output() 
    together effectively - they conflict.
    Pattern: tools_llm (use tools) → formatter → structured data
    """
    
    if hasattr(tool_result, 'content'):
        content = tool_result.content
    else:
        content = str(tool_result)
        
    current_stage = state.get("stage", "initial")
    business_info = state.get("business_info", "None")
    existing_competitors = state.get("competitors", [])
    competitors_query = state.get("competitors_query", {})
    competitors_report = state.get("competitors_report", {})
    
    STRUCT_SYSTEM_PROMPT = f"""
    You are a data extractor. Extract information from tool/search results and format into the required schema.
    EXTRACTION RULES:
    1. Extract ONLY factual information present in the results
    2. For competitors: extract as list of company/person names
    3. For reports: extract findings as dictionary {{competitor: report}}
    4. Do NOT invent, hallucinate, or add information not in the source
    5. If data is missing, leave field empty/null
    6. Preserve exact names and facts from source
    CURRENT CONTEXT:
    - Stage: {current_stage}
    - Business: {business_info}
    - Known Competitors: {existing_competitors}
    - Competitors Queries: {competitors_query}
    - Competitors Reports: {competitors_report}
    OUTPUT: Return structured data matching the schema. No markdown, no explanations.
    """

    structured_result = struct_llm.invoke([
        SystemMessage(content=STRUCT_SYSTEM_PROMPT),
        HumanMessage(content=f"Extract and structure this:\n\n{content}")
    ])

    return structured_result


In [ ]:
# Nodes

def get_competitors(state: State) -> State:
    """Get competitors from the business info with error handling"""
    try:
        business_info = state.get("business_info", "")
        
        if not business_info:
            return {
                "messages": [AIMessage(content="⚠️ I need your business information first to find competitors.")],
                "stage": INITIAL
            }
        
        system_prompt = f"""You are a competitive intelligence researcher with web search tools.
        Business Info: {business_info}
        TASK: Find the TOP 3-5 direct competitors for this business.
        If there are existing competitors in the competitors field make more research on competitors in the same industry/niche and reshuffle the list. Note max length 5 competitors
        Use browser tools to search for competitors in the same industry/niche.
        Requirements:
        - Competitors can be companies OR individuals
        - Only list REAL names and url
        - Focus on direct competitors
        Return the competitor names clearly."""
        
        result = struct_llm.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"Find competitors for: {business_info}")
        ])

        # result = formatter(result, state)
        
        competitors = result.competitors if hasattr(result, 'competitors') and result.competitors else []
        
        if not competitors:
            return {
                "messages": [AIMessage(content="⚠️ I couldn't find competitors automatically. Could you tell me who your main competitors are?")],
                "stage": GET_COMPETITORS
            }
        
        return {
            "messages": [AIMessage(content=result.response if hasattr(result, 'response') else f"Found competitors: {', '.join(competitors)}")],
            "competitors": competitors,
            "stage": CONFIRM_COMPETITORS
        }
        
    except Exception as e:
        print(f"Error in get_competitors: {e}", flush=True)
        return {
            "messages": [AIMessage(content="⚠️ I encountered an error searching for competitors. Could you tell me who your main competitors are?")],
            "stage": GET_COMPETITORS
        }

def get_competitors_prompt(state: State) -> State:
    """Generate search queries with error handling"""
    try:
        business_info = state.get("business_info", "")
        competitors = state.get("competitors", [])
        
        if not competitors:
            return {
                "messages": [AIMessage(content="⚠️ No competitors to research yet.")],
                "stage": GET_COMPETITORS
            }
        
        system_prompt = f"""You are a research strategist.
        Business Info: {business_info}
        Competitors: {', '.join(competitors)}
        TASK: Generate detailed search queries for EACH competitor considering the business info. using the browser tools"""
        
        result = struct_llm.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"Generate search queries for: {', '.join(competitors)}")
        ])

        # result = formatter(result, state)
        
        queries = result.competitors_query if hasattr(result, 'competitors_query') and result.competitors_query else {}
        
        if not queries:
            queries = {comp: f"{comp} products services pricing features" for comp in competitors}
        
        return {
            "messages": [AIMessage(content=result.response if hasattr(result, 'response') else "Search queries generated")],
            "competitors_query": queries,
            "stage": GET_REPORT
        }
        
    except Exception as e:
        print(f"Error in get_competitors_prompt: {e}", flush=True)
        queries = {comp: f"{comp} business overview" for comp in state.get("competitors", [])}
        return {
            "messages": [AIMessage(content="⚠️ Generated basic search queries.")],
            "competitors_query": queries,
            "stage": GET_COMPETITORS_PROMPT
        }

def get_report(state: State) -> State:
    """Get detailed reports with error handling"""
    try:
        competitors_query = state.get("competitors_query", {})
        competitors = state.get("competitors", [])
        
        if not competitors_query or not competitors:
            return {
                "messages": [AIMessage(content="⚠️ Missing competitor information to generate reports.")],
                "stage": INITIAL
            }
        
        message = f"""You are a competitive intelligence researcher.
        Use browser tools to research each competitor.
        Competitors Queries: {competitors_query}
        Return comprehensive information about each competitor."""
        
        result = struct_llm.invoke([
            SystemMessage(content=message),
            HumanMessage(content="Research all competitors")
        ])
        # result = formatter(result, state)
        reports = result.competitors_report if hasattr(result, 'competitors_report') and result.competitors_report else {}
        
        if not reports:
            reports = {comp: f"Research data for {comp}" for comp in competitors}
        
        return {
            "messages": [AIMessage(content=result.response if hasattr(result, 'response') else "Research complete")],
            "stage": FINAL,
            "competitors_report": reports,
        }
        
    except Exception as e:
        print(f"Error in get_report: {e}", flush=True)
        return {
            "messages": [AIMessage(content="⚠️ Error generating reports. Let me try to provide a basic analysis with available information.")],
            "stage": FINAL,
            "competitors_report": {comp: "Limited data available" for comp in state.get("competitors", [])}
        }
    
def planner(state: State) -> State:
    """Plan the research with error handling"""
    try:
        system_prompt = f"""
        You handle competitive business analysis. Be concise, conversational, and professional.
        GUIDE:
        - Greet politely if greeted.
        - If user asks questions about competitors, adding/removing competitors, or comparisons → ALWAYS process naturally.
        - If user says "remove this competitor", "add this competitor", "what about X competitor" → Update the competitors list accordingly.
        - If question or message is COMPLETELY unrelated to business/competitors → reply: "I only handle competitor analysis."
        - Always focus on understanding their business and comparing competitors.
        WORKFLOW:
        1. **Greeting:** Welcome user and mention you handle competitor analysis.
        2. **Business Info:** 
           - Ask 2-3 clear questions if missing.
           - Critical: Understand their business type, scale, market/region, the country they are operating in and what makes them unique.
           - Ask targeted questions based on what you already know, not generic ones.
           - Ask follow-ups if vague.
           - When clear → Summarize in business_info field, move forward.
        3. **Competitors:**
           - If business info is clear but competitors are missing → set stage='get_competitors'.
           - When stage=confirm_competitors → Ask user to confirm/modify the competitors list.
           - You can ask user if they wish you to help them add more competitors or remove competitors at this stage.
           - User can add more competitors or remove competitors at this stage.
           - ONLY after user explicitly confirms (e.g., "yes", "looks good", "proceed", "continue") → set stage='get_competitors_prompt'.
           - If user modifies list → update competitors field and ask for confirmation again.
           - if a user ask you to add more by yourself set stage=get_competitors
           - after user confirms the competitors list set stage=get_competitors_prompt
       4. **Final Report (READ-ONLY - You cannot SET this stage):**
           - You will ONLY see stage=final after the system completes research
           - When you see stage=final and competitors_report exists → IMMEDIATELY generate the complete MARKDOWN report
           - DO NOT manually set stage=final yourself
           - The workflow automatically moves: get_competitors_prompt → get_report → final
           - Your job at final: generate the report with:
               * Executive summary (2-3 sentences)
               * Your business overview
               * Each competitor: strengths, weaknesses, differentiators
               * Comparison table (You vs Competitors)
               * Recommendations: opportunities, differentiation, action plan (3 items)
               * Key takeaways (3 points)
           - Use headers (##), tables (|), bullets (-), emojis for sections
           - Base everything on ACTUAL research data from competitors_report
           - After generating the report set stage=initial
        CRITICAL RULES:
        - ALWAYS handle competitor modification requests (add/remove/change).
        - When user confirms competitors → set stage=get_competitors_prompt
        - Stay on topic and professional.
        - Don't change stage until confirmed or task is ready.
        - Be brief, clear, and business-focused.
        - When stage=final, generate report IMMEDIATELY without asking and then set stage=initial.
        CURRENT STATE:
        - Stage: {state.get("stage", "NONE")}
        - Business Info: {state.get("business_info", "NONE")}
        - Competitors: {", ".join(state.get("competitors", [])) or "NONE"}
        """
        
        messages = [
            SystemMessage(content=system_prompt),
            *state.get("messages", []),
        ]
        
        if state.get("stage") == GET_COMPETITORS_PROMPT or state.get("stage") == GET_REPORT:
            return state
        
        result = struct_llm.invoke(messages)
        
        return {
            "messages": [AIMessage(content=result.response if hasattr(result, 'response') else "Let me help you with competitor analysis.")],
            "stage": result.stage if hasattr(result, 'stage') and result.stage else INITIAL,
            "business_info": result.business_info if hasattr(result, 'business_info') and result.business_info else state.get("business_info", ""),
            "competitors": result.competitors if hasattr(result, 'competitors') and result.competitors else state.get("competitors", []),
            "competitors_query": result.competitors_query if hasattr(result, 'competitors_query') and result.competitors_query else state.get("competitors_query", {}),
            "competitors_report": result.competitors_report if hasattr(result, 'competitors_report') and result.competitors_report else state.get("competitors_report", {})
        }
        
    except Exception as e:
        print(f"Error in planner: {e}", flush=True)
        return {
            "messages": [AIMessage(content="⚠️ I encountered an issue. Let's start fresh - tell me about your business.")],
            "stage": INITIAL,
            "business_info": state.get("business_info", ""),
            "competitors": state.get("competitors", []),
            "competitors_query": state.get("competitors_query", {}),
            "competitors_report": state.get("competitors_report", {})
        }

In [ ]:
# Node conditions

def node_conditions(state:State):
    if state.get("stage") == GET_COMPETITORS:
        return GET_COMPETITORS
    elif state.get("stage") == GET_COMPETITORS_PROMPT:
        return GET_COMPETITORS_PROMPT
    elif state.get("stage") == GET_REPORT:
        return GET_REPORT
    else:
        return "END"

def tools_routing(state:State):
    if state.get("stage") == GET_COMPETITORS:
        return GET_COMPETITORS
    elif state.get("stage") == GET_COMPETITORS_PROMPT:
        return GET_COMPETITORS_PROMPT
    elif state.get("stage") == GET_REPORT:
        return GET_REPORT
        

In [ ]:
# Graph

graph_builder = StateGraph(State)

graph_builder.add_node("get_competitors", get_competitors)
graph_builder.add_node("get_competitors_prompt", get_competitors_prompt)

graph_builder.add_node("get_report", get_report)
graph_builder.add_node("tools", ToolNode(tools=tools))
graph_builder.add_node("planner", planner)


In [ ]:
# Edges

graph_builder.add_edge(START, "planner")
graph_builder.add_conditional_edges("planner", node_conditions, {GET_COMPETITORS: "get_competitors", GET_COMPETITORS_PROMPT: "get_competitors_prompt", GET_REPORT: "get_report", "END": END})
graph_builder.add_conditional_edges("get_competitors", tools_condition, {"tools": "tools", END: "planner"})
graph_builder.add_conditional_edges("get_competitors_prompt", tools_condition, {"tools": "tools", END: "planner"})
graph_builder.add_conditional_edges("get_report", tools_condition, {"tools": "tools", END: "planner"})
graph_builder.add_conditional_edges("tools", tools_routing, {GET_COMPETITORS: "get_competitors", GET_COMPETITORS_PROMPT: "get_competitors_prompt", GET_REPORT: "get_report"})

In [ ]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:


def reset_conversation():
    """Reset to new conversation"""
    global THREAD_ID
    THREAD_ID = str(uuid.uuid4())
    return None, "" 

def compat_message(message, owner):
    return {"role": owner, "content": message}

async def chat(message, history):
    try:
        yield [*history, compat_message(message=message, owner="user"), compat_message(message="✨ Thinking ...", owner="assistant")]
        if not message or message.strip() == "":
            yield [*history, compat_message(message=message, owner="user"), compat_message(message="Please enter a message.", owner="assistant")]
        
        config = {"configurable": {"thread_id": THREAD_ID}}
        state = {"messages": [HumanMessage(content=message)]}
        
        result = None
        async for event in graph.astream(state, config, stream_mode="values"):
            result = event
            stage = event.get("stage", "")
            if stage:
                beautified_text = beautify_output.get(stage, "")
                if beautified_text:
                    yield [*history, compat_message(message=message, owner="user"), compat_message(message=beautified_text, owner="assistant")]
        
        if not result or "messages" not in result or not result["messages"]:
            yield [*history, compat_message(message=message, owner="user"), compat_message(message="⚠️ I didn't receive a proper response. Please try again.", owner="assistant")]
        
        yield [*history, compat_message(message=message, owner="user"), compat_message(message=result["messages"][-1].content, owner="assistant")]
        
    except Exception as e:
        print(f"Chat error: {e}", flush=True)
        yield [*history, compat_message(message=message, owner="user"), compat_message(message="⚠️ Something went wrong. Please try again or start a new conversation.", owner="assistant")]


In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="🔍 Competitor Analysis Agent") as app:
    gr.Markdown("""
    # 🔍 Competitive Analysis Agent
    
    Get detailed insights about your competitors and actionable recommendations for your business.
    
    **How it works:**
    1. Tell me about your business
    2. I'll find and research your competitors
    3. Get a comprehensive analysis report
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                label="Chat",
                height=500,
                type="messages",  # Better message display
                show_label=False
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    label="Your message",
                    placeholder="Tell me about your business...",
                    show_label="Your message",
                    submit_btn=" Send ",
                    scale=4 
                )
            
            with gr.Row():
                clear_btn = gr.Button("🔄 New Conversation", variant="secondary")
                
        with gr.Column(scale=1):
            gr.Markdown("### 💡 Tips")
            gr.Markdown("""
            - Be specific about your business
            - Mention your target market
            - Include your location/region
            - Tell me your business scale
            
            ### 📝 Example Questions
            - "I run a freelance web design business"
            - "Help me analyze my competitors"
            - "I'm a solo consultant in Nigeria"
            """)
    
    # Event handlers
    msg.submit(chat, [msg, chatbot], chatbot, queue=True)
    msg.submit(lambda:"", outputs=msg)
    clear_btn.click(reset_conversation, None, [chatbot, msg])

app.launch()